# 1.1 🚀 Initialization and Configuration

* In this section, necessary libraries are imported, and the Semantic Kernel is configured with Azure OpenAI credentials loaded from a configuration file.

In [132]:
#!import config/Settings.cs

In [133]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta8"

Installed Packages Microsoft.SemanticKernel, 1.0.0-beta8

In [134]:
using Microsoft.SemanticKernel;

var builder = new KernelBuilder();

In [135]:
using Kernel = Microsoft.SemanticKernel.Kernel;

var (model1, azureEndpoint1, apiKey1, gpt35TurboServiceId) = Settings.LoadFromFile(model: "gpt-35-turbo");
builder.WithAzureOpenAIChatCompletionService(model1, azureEndpoint1, apiKey1, serviceId: gpt35TurboServiceId);

var (model2, azureEndpoint2, apiKey2, gpt4ServiceId) = Settings.LoadFromFile(model: "gpt-4-32k");
builder.WithAzureOpenAIChatCompletionService(model2, azureEndpoint2, apiKey2, serviceId: gpt4ServiceId);

var kernel = builder.Build();

# 1.2 🚀 Semantic Function for Short Intent Extraction
* This section focuses on setting up the Semantic Function for extracting a concise user intent using a specific prompt.

In [136]:
using Microsoft.SemanticKernel.TemplateEngine;
using Microsoft.SemanticKernel.TemplateEngine.Basic;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;
using Microsoft.SemanticKernel.AI;

var aiRequestSettings = new AIRequestSettings 
{
    ExtensionData = new Dictionary<string, object> { { "api-version", "2023-03-15-preview" } },
    ServiceId = gpt35TurboServiceId
};

In [137]:
var input = "I want to find top-10 books about world history";

In [138]:
// Chat with the Bot
// Create a simple prompt for the semantic function

string skPrompt = @"ChatBot: How can I help you?
User: {{$input}}

---------------------------------------------

The intent of the user in 25 words or less: ";

var getShortIntentFunction = kernel.CreateSemanticFunction(skPrompt, requestSettings: aiRequestSettings);

var intentResult = await kernel.RunAsync(input, getShortIntentFunction);

Console.WriteLine(intentResult);

User wants recommendations for the top 10 books about world history.


# 1.3 🚀 Semantic Function for Conversational Chat

* This section defines the Semantic Function for engaging in a more extended conversation with the chatbot, initializing the context, and executing the function.

In [139]:
skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give detailed answer or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$userInput}}
ChatBot:";

In [140]:
// Chat with the Bot

var aiRequestSettings = new AIRequestSettings 
{
    ExtensionData = new Dictionary<string, object> { { "api-version", "2023-03-15-preview" } },
    ServiceId = gpt4ServiceId
};

var chatFunction = kernel.CreateSemanticFunction(skPrompt, "fn", "pn", requestSettings: aiRequestSettings);
var context = kernel.CreateNewContext();

var history = "";
context.Variables["history"] = history;

var userInput = "Hi, I'm looking for suggestions of best sellers about world history";
context.Variables["userInput"] = userInput;

var bot_answer = await chatFunction.InvokeAsync(context);

In [141]:
// Update the history with the output and set this as the new input value for the next request

history += $"\nUser: {userInput}\nSuggestions: {bot_answer.GetValue<string>()}\n";
context.Variables.Update(history);

Console.WriteLine(context);


User: Hi, I'm looking for suggestions of best sellers about world history
Suggestions: Sure, here are some best-selling books about world history that you might find interesting:

1. "Sapiens: A Brief History of Humankind" by Yuval Noah Harari
2. "Guns, Germs, and Steel: The Fates of Human Societies" by Jared Diamond
3. "A People's History of the United States" by Howard Zinn
4. "The Silk Roads: A New History of the World" by Peter Frankopan
5. "The History of the World in Six Glasses" by Tom Standage
6. "The World: A Brief Introduction" by Richard Haass
7. "The Penguin History of the World" by J.M. Roberts
8. "The Story of Civilization" by Will Durant
9. "The Lessons of History" by Will and Ariel Durant
10. "The Rise and Fall of the Great Powers" by Paul Kennedy

Remember to check the availability of these books in your local library or bookstore. Happy reading!



# 1.4 🚀 Interactive Chat Function

* In this final section, an interactive chat function is defined, and several interactions with the chatbot are simulated. The complete chat history and the final user input are then printed. *

In [142]:
// Chatting more)

Func<string, Task> Chat = async (string input) => {
    // Save new message in the context variables
    context.Variables["userInput"] = input;

    // Process the user message and get an answer
    var answer = await chatFunction.InvokeAsync(context);

    // Append the new interaction to the chat history
    history += $"\nUser: {input}\nSuggestions: {answer.GetValue<string>()}\n"; 
    context.Variables["history"] = history;
    
    // Show the response
    Console.WriteLine(context);
};

In [143]:
await Chat("I would like a non-fiction book suggestion about Greece history. Please only list one book.");

Sure, I would recommend "The History of the Peloponnesian War" by Thucydides. It's a classic work of ancient Greek history.


In [144]:
await Chat("that sounds interesting, what are some of the topics I will learn about?");

"The History of the Peloponnesian War" by Thucydides is a detailed historical account of the Peloponnesian War, which was fought between the city-states of Athens and Sparta, and their respective allies, in ancient Greece. Here are some of the topics you will learn about:

1. The causes and origins of the Peloponnesian War.
2. The strategies, tactics, and battles of the war.
3. The political and social structures of Athens and Sparta.
4. The impact of the war on the people of Athens and Sparta.
5. The role of key figures and leaders during the war, such as Pericles and Alcibiades.
6. The eventual defeat of Athens and the consequences of the war on ancient Greece.
7. Thucydides' own analysis and interpretation of the events.

It's worth noting that Thucydides' account is considered one of the earliest and most important works of history and political theory. It offers valuable insights into the nature of power, war, and human behavior.


In [145]:
await Chat("Which topic from the ones you listed do you think the most popular?");

The most popular topic among readers often tends to be the strategies, tactics, and battles of the war. Many readers find the detailed descriptions of military engagements, the strategies employed by both sides, and the tactical decisions made by leaders to be particularly fascinating. However, the popularity of a topic can vary greatly depending on the reader's personal interests. Some might be more interested in the political and social structures of Athens and Sparta, while others might be drawn to the personal stories and roles of key figures during the war.


In [146]:
Console.WriteLine(context.Variables["history"]);


User: Hi, I'm looking for suggestions of best sellers about world history
Suggestions: Sure, here are some best-selling books about world history that you might find interesting:

1. "Sapiens: A Brief History of Humankind" by Yuval Noah Harari
2. "Guns, Germs, and Steel: The Fates of Human Societies" by Jared Diamond
3. "A People's History of the United States" by Howard Zinn
4. "The Silk Roads: A New History of the World" by Peter Frankopan
5. "The History of the World in Six Glasses" by Tom Standage
6. "The World: A Brief Introduction" by Richard Haass
7. "The Penguin History of the World" by J.M. Roberts
8. "The Story of Civilization" by Will Durant
9. "The Lessons of History" by Will and Ariel Durant
10. "The Rise and Fall of the Great Powers" by Paul Kennedy

Remember to check the availability of these books in your local library or bookstore. Happy reading!

User: I would like a non-fiction book suggestion about Greece history. Please only list one book.
Suggestions: Sure, I wou

In [147]:
Console.WriteLine(context.Variables["userInput"]);

Which topic from the ones you listed do you think the most popular?


# 1.5 🚀 Switching between models
* Call HuggingFace model by name 1

In [152]:
#r "nuget: Microsoft.SemanticKernel.Connectors.AI.HuggingFace, 1.0.0-beta8"

Installed Packages Microsoft.SemanticKernel.Connectors.AI.HuggingFace, 1.0.0-beta8

In [169]:
using Kernel = Microsoft.SemanticKernel.Kernel;

var (hfModel, hfEndpoint, hfApiKey, huggingFaceServiceId) = Settings.LoadFromFile(model: "meta-llama/Llama-2-7b-hf");

#pragma warning disable SKEXP0020
builder.WithHuggingFaceTextCompletionService(hfModel, apiKey: hfApiKey);

var kernel = builder.Build();

string skPrompt = @"What drives climate change and its impact on the environment?";

var hfFunction = kernel.CreateSemanticFunction(skPrompt);
var hfResult= await kernel.RunAsync(input, hfFunction);

Console.WriteLine(hfResult.GetValue<string>());

Error: Microsoft.SemanticKernel.Diagnostics.HttpOperationException: Response status code does not indicate success: 400 (Bad Request).
 ---> System.Net.Http.HttpRequestException: Response status code does not indicate success: 400 (Bad Request).
   at System.Net.Http.HttpResponseMessage.EnsureSuccessStatusCode()
   at HttpClientExtensions.SendWithSuccessCheckAsync(HttpClient client, HttpRequestMessage request, HttpCompletionOption completionOption, CancellationToken cancellationToken)
   --- End of inner exception stack trace ---
   at HttpClientExtensions.SendWithSuccessCheckAsync(HttpClient client, HttpRequestMessage request, HttpCompletionOption completionOption, CancellationToken cancellationToken)
   at HttpClientExtensions.SendWithSuccessCheckAsync(HttpClient client, HttpRequestMessage request, CancellationToken cancellationToken)
   at Microsoft.SemanticKernel.Connectors.AI.HuggingFace.TextCompletion.HuggingFaceTextCompletion.ExecuteGetCompletionsAsync(String text, Cancel